In [ ]:
import sys
import os
import os.path 
sys.path.append(r"C:\Users\15593\Documents\repos\Spectral-Utilities")
from SpectralUtilities import HSI_Model, HDRprocess, GPGGAParser

def get_hsi_data(dataDir:str) -> tuple[str,str]:
    captureList = [root for root, dirnames, filenames in os.walk(dataDir) if  "capture" in root  ]
    imgNames = HDRprocess.get_hsi_folder_names(dataDir,not_std=True)
    hdrList = [HDRprocess.get_hdr_file_path(cap) for cap in captureList]
    navList = [f"{hdr[:-3]}nav" for hdr in hdrList]
    return hdrList, navList, imgNames

In [ ]:
# Change the data directory to top-level folder containing parent captures
dataDir = r"Z:\Data\Gallo_berrycompo\Specim_images\1007"
# Where things will get saved to
savePath = r"Z:\Data\Gallo_berrycompo\Specim_images\1007_Georef_Tif"

In [ ]:
#Gather the paths for the data
hdrList, navList, imgNames = get_hsi_data(dataDir=dataDir)

In [ ]:
for idx,img in enumerate(imgNames):
    bandSavePath = os.path.join(savePath,img)
    rgbSavePath = os.path.join(bandSavePath,"rgb")
    if "_nfl1" in img:
        southUp = True
    else:
        southUp = False

    if not os.path.isdir(bandSavePath):
        os.mkdir(bandSavePath)
    if not os.path.isdir(rgbSavePath):
        os.mkdir(rgbSavePath)
    Data_HSI = HSI_Model(path_hcube=hdrList[idx],imgName=img,dataset="pushbroom",nav_file=navList[idx])
    # create an rgb
    Data_HSI.set_rgb_by_wv_index(r=25,g=54,b=68,rot=True)
    Data_HSI.save_rgb(rgbSavePath)
    for bandIdx, band in enumerate(Data_HSI.wv):
        Data_HSI.georef_pushbroom_band(savePath=bandSavePath,band=bandIdx,diagonalExtent=2,dOffset=1,rot=southUp)